### Troubles with $0^t$

Let's evaluate the following integral in Sympy:

$$\int x^t\, \mathrm{d}x = \frac{x^{t+1}}{t+1} + C,\, t\neq -1.$$


In [8]:
from sympy import Symbol, integrate

x = Symbol('x')
t = Symbol('t')
f = x**t

integrate(f, x)

Piecewise((x**(t + 1)/(t + 1), Ne(t, -1)), (log(x), True))

So far so good. But see what happens if we try to calculate its definite form on $[0, 1]$. With a pencil and a piece of paper you would obtain

$$\int_0^1 x^t\,\mathrm{d}x = \frac{1}{t+1},\, t\gt -1.$$

However, here is what Sympy returns:

In [9]:
integrate(f, (x, 0, 1))

Piecewise((-0**(t + 1)/(t + 1) + 1/(t + 1), (t > -oo) & (t < oo) & Ne(t, -1)), (oo, True))

We have been given a kind of too wild piecewise expression. The first condition can be reduced to just $t\neq-1$.

If we specify that $t$ lies in real domain, Sympy does simplify the condition. Why? I don't know.

In [6]:
t = Symbol('t', real=True)
f = x**t
integrate(f, (x, 0, 1))

Piecewise((-0**(t + 1)/(t + 1) + 1/(t + 1), Ne(t, -1)), (oo, True))

Moreover, we can see some pretty ugly leftover,

$$h(t) := -\frac{0^{t+1}}{t+1}$$

that can be rewritten as follows:

$$\begin{equation*}
h(t) = \left\{
        \begin{array}{ll}
            0 & \quad t \gt -1 \\
            \mathrm{NaN} & \quad t \leq -1
        \end{array}
    \right.
\end{equation*}$$

It is safe to claim that
* $h(-1) = -\frac{0^0}{0}$ is undefined
* $h(t\lt -1) = \frac{1}{0}$ is undefined.

However, Sympy developers have put $\lim_{x\to 0} 1/x \equiv \infty = \lim_{x\to 0^+} 1/x \neq \lim_{x\to 0^-} 1/x = -\infty$. If we turn a blind eye, this choice (omitting the whole negative branch) is justifiable especially for natural scientists using Sympy (e.g. in case of gravitational potential $U(r) \propto -1/r$, we don't need to worry about negative distances and we "don't approache $0$ from below"). 

In [4]:
from sympy import limit
limit(1/x, x, 0)

oo

Hence we are allowed to write

$$\begin{equation}
h(t) = 
        \begin{cases}
            0 & \quad t \gt -1 \\
            \infty & \quad t \leq -1
        \end{cases}
\end{equation}$$

Combining it with $1/(t+1)$ we obtain in Sympy context

$$\begin{equation*}
I(t) = \left\{
        \begin{array}{ll}
            0 + \frac{1}{t+1}& \quad t \gt -1 \\
            \infty\equiv\mathrm{NaN} & \quad t \leq -1
        \end{array}
    \right.
\end{equation*}$$

Sympy does return a correct answer if we restrict $t$ to take only non-negative values (I am afraid that is the maximum we can do now): 

In [5]:
x = Symbol('x')
t = Symbol('t', nonnegative=True)
f = x**t

integrate(f, (x, 0, 1))

1/(t + 1)

### Other suspicious results

Since Sympy seems to overlook cases involving $0^t$ and leaves this expression untreated, we might encounter other issues. For example, let's ask Sympy where the function $h(t)$ is continuous. We expect the interval $(-1, \infty)$ but instead, we are given:

In [6]:
from sympy import S, limit
from sympy.calculus.util import continuous_domain
from sympy.calculus.singularities import singularities

t = Symbol('t', real=True)
h = -0**(t+1)/(t+1)
continuous_domain(h, t, S.Reals)

Union(Interval.open(-oo, -1), Interval.open(-1, oo))

And indeed, Sympy seems to check only the denominator: 

In [7]:
continuous_domain(0**t, t, S.Reals)

Reals

The same is true for singularities. Sympy will check the denominator and is done:

In [9]:
singularities(h, t)

{-1}

There are uncountable many singularites: $(-\infty, -1]$. From this, we can guess that Sympy will tell us that $0^t$ has no singularities whatsoever:

In [10]:
singularities(0**t, t)

EmptySet

Simply put: $0^t$ must be treated in Sympy and the corresponding functions should be modified accordingly.

[Sympy issue opened](https://github.com/sympy/sympy/issues/26949).